In [ ]:
#pip install openpyxl
#pip install mano

In [ ]:
from platform import python_version
import sys
import os
import pandas as pd
import mano
import time
import mano.sync as msync
import numpy as np

In [ ]:
print(python_version())
print(sys.executable)

In [ ]:
#proj_shared_dir = "/Users/martakaras/Dropbox/Beiwe Resources - for Accelerometry Analysis"
proj_shared_dir = "/Users/mak5182/Dropbox/Beiwe Resources - for Accelerometry Analysis"
beiwe_data_raw_dir  = os.path.join(proj_shared_dir, "data_beiwe_raw")
beiwe_data_raw_dir

### Set up the keyring

In [ ]:
#sys.path.insert(0, '/Users/martakaras/Documents/data_beiwe_settings/')
sys.path.insert(0, '/Users/mak5182/Documents/data_beiwe_settings/')
import keyring_studies_MK
Keyring = mano.keyring(None)

### Prepare subject ID list

In [ ]:
file_name = "smart_wearables_start_end_dates_clean.csv"
file_path = os.path.join(proj_shared_dir, "data_participants_other_processed", file_name)
SMARTW_user_id_df = pd.read_csv(file_path)

In [ ]:
user_id_list = SMARTW_user_id_df['beiwe_id'].tolist()
# remove nan
user_id_list = [x for x in user_id_list if str(x) != 'nan']
# append one additional beiwe_id 
# (NOTE: actual value replaced with "foobar" for the public distribution)
user_id_list.append("foobar")
# how many Beiwe ID
print(len(user_id_list))
# 50

### Download survey and identifiers data

In [ ]:
# study ID
# (NOTE: actual value replaced with "foobar2" for the public distribution)
study_id = 'foobar2'
data_streams = ['identifiers', 'survey_answers', 'survey_timings']

In [ ]:
user_id_list[:5]

In [ ]:
# list to store elapsed time and other stats
t_diff_vec = []
status_vec = []
error_user_id_vec = []

# run download
for i in range(len(user_id_list)):
#for i in range(10, len(user_id_list), 1):
    user_id = user_id_list[i]
    t_start = time.time()
    try:
        zf = msync.download(Keyring, study_id, user_id, data_streams = data_streams)
        zf.extractall(beiwe_data_raw_dir)
        status_vec.append(1)
        print("SUCCESS")
    except BaseException as e:
        print(str(e))
        status_vec.append(0)
        error_user_id_vec.append(user_id)
        print("FAIL")
    # append elapsed time info 
    t_end   = time.time()
    t_diff  = t_end - t_start
    t_diff_vec.append(t_diff)
    summary_log = "i = " + str(i) + ", STUDY = " + study_id + ", USER = " + user_id + ", TIME ELAPSED = " + str(np.round(t_diff, 1)) + "s"
    print(summary_log)